Sesion 2
- Mate de Log
- Modelo de regresión logística: diseño, supuestos, estimación y evaluación, interpretación - BREVE
- Mate de ANOVA
- MANOVA: diseño, supuestos, estimación y evaluación, interpretación